# Imports
Import required libraries

In [19]:
from __future__ import print_function, division
import os
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import sklearn as sklearn
from sklearn.cross_validation import train_test_split, KFold, cross_val_predict
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from numpy import random
import sqlite3
%matplotlib inline

# 1) Description of the dataset
The initial database has four tables: Countries, leagues, teams, team_atts, matches, players, player_atts.

Countries - The country_id for each match (which country the match took place in)

Leagues - The league_id for each match (as each country only has one league in this set, this is identical to country_id and not needed).

Teams - Contains the id, team_api_id, team_fifa_api_id, and name of each team. The team_fifa_api_id is what the id of each team is in the FIFA games (where player/team statistics are pulled from).

Team_atts - Contains attributes about each team: their playstyle, offense, defense, etc.

Matches - This contains the bulk of the data that we want. Contains information about: who the home/away team are, which league the game took place in, which season, the data, player statistics, and betting data.

Players - Contains information about each player: their id, age, team, stature

Player_atts - Has information on the attributes of each player: their skills, strengths, weaknesses, but most importantly their player rating.

--A side note, teams and players contain both a FIFA and non-FIFA key. Matches are joined to them by non-FIFA key.--

We will now read-in the tables from the sqlite database.

In [57]:
with sqlite3.connect('database.sqlite') as con:
    countries = pd.read_sql_query("SELECT * from Country", con)
    matches = pd.read_sql_query("SELECT * from Match", con)
    players = pd.read_sql_query("SELECT * from Player", con)
    player_atts = pd.read_sql_query("SELECT * from Player_Attributes", con)
    team_atts = pd.read_sql_query("SELECT * from Team_Attributes", con)
    teams = pd.read_sql_query("SELECT * from Team", con)


For now, we will currently only be examining betting data from bet365. Many of the betting sites included in the dataset are missing betting data on a large portion of the matches and bet365 has by far the most.

We will also drop attributes that will not be useful for what we want yet; these attributes are generally ingame statistics of the matches in general.

`league_id` is also dropped because we are not using the leagues table.

In [58]:
# Drops unneeded attributes
atts_to_drop = ['BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA', 'LBH', 'LBD', 'LBA', 'GBD',
               'PSH', 'PSD', 'PSA', 'WHH', 'WHA', 'WHD', 'SJH', 'SJD', 'SJA', 'VCH',
               'VCD', 'VCA', 'GBH', 'GBA', 'BSH', 'BSD', 'BSA', 'shoton', 'goal', 'shotoff',
               'foulcommit', 'card', 'cross', 'corner', 'possession', 'league_id']
matches = matches.drop(atts_to_drop, axis=1) 

# Drops player formation values
matches = matches.drop(matches.columns[10:54], axis=1)

# Drops null values
matches = matches.dropna()

# Replace home_team_goal and away_team_goal with home_win (0 = no, 1 = draw, 2 = yes)
def find_winner(row):
    if row['home_team_goal'] > row['away_team_goal']:
        return 2
    if row['home_team_goal'] == row['away_team_goal']:
        return 1
    if row['home_team_goal'] < row['away_team_goal']:
        return 0
    
matches.insert(9, 'match_result', -1)
matches['match_result'] = matches.apply(lambda x: find_winner(x), axis = 1)
matches = matches.drop(['home_team_goal', 'away_team_goal'], axis = 1)


In [59]:
england_matches = matches[(matches['country_id'] == 1729)]
france_matches = matches[(matches['country_id'] == 4769)]
germany_matches = matches[(matches['country_id'] == 7809)]
italy_matches = matches[(matches['country_id'] == 10257)]
netherlands_matches = matches[(matches['country_id'] == 13274)]
spain_matches = matches[(matches['country_id'] == 21518)]

We will take the dates and append the players' player_id with them (to make it easier to process), as player_atts contains different player rating values for each date they play.

In [60]:
players = ['home_player_1', 'home_player_2', 'home_player_3', 'home_player_4', 'home_player_5',
          'home_player_6', 'home_player_7', 'home_player_8', 'home_player_9', 'home_player_10', 'home_player_11',
          'away_player_1', 'away_player_2', 'away_player_3', 'away_player_4', 'away_player_5', 'away_player_6',
          'away_player_7', 'away_player_8', 'away_player_9', 'away_player_10', 'away_player_11']

for col in players:
    matches[col] = matches[col].map(str) + ' ' + matches['date']

player_atts['player_api_id'] = player_atts['player_api_id'].map(str) + ' ' + player_atts['date']

In [61]:
player_atts.head()

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942 2016-02-18 00:00:00,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942 2015-11-19 00:00:00,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942 2015-09-21 00:00:00,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942 2015-03-20 00:00:00,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942 2007-02-22 00:00:00,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0


Now, we will replace player_ids with their player rating values in FIFA.

In [ ]:
matches[players] = matches[players].applymap(lambda x: player_atts.loc[(player_atts['player_api_id'] == x)]['overall_rating'])

In [ ]:
player_atts.head()
player_atts.loc[(player_atts['player_api_id'] == 505942)]

# 3) Visualizing the data-set

In [ ]:
multipred = england_matches.drop(['date','season','B365H', 'B365A', 'B365D', 'match_result', 'country_id', 'id', 'match_api_id'], axis=1)
multitarget = england_matches['B365H']
kf = KFold(multitarget.size, n_folds = 10, shuffle= True, random_state = 0) 
lr =  LinearRegression(normalize = True)
result = cross_val_predict(lr, multipred, multitarget, cv = kf)

slope, intercept, r_value, p_value, std_err = stats.linregress(multitarget, result)

#print("R^2: ", r_value**2)
#print("RMSE: ", mean_squared_error(multitarget, result)**(1/2.0))
#print("MAE: ", mean_absolute_error(multitarget, result))
#print("CC: ", np.corrcoef(result, multitarget)[1][0])

# 3) Ranking the Leagues by Ease of Prediction
Fans of soccer know that certain leagues are easier to predict than others. For example, the Spanish La Liga is notoriously top-heavy, with Real Madrid and Barcelona dominating the league for the last few decades. We will now use 

In [ ]:
england_matches.columns.values